In [20]:
import sys
import pandas as pd
import random
from collections import Counter
random.seed(0)

In [22]:
main_dir ="/u/home/b/briscoel/project-halperin/MicroBatch/data/"
script_folder= "/u/home/b/briscoel/project-halperin/MicroBatch/RevisionSequence/"
local = True

sys.argv = [ "Thomasr_complete_otu","rel",""]

if local:
    main_dir = "/Users/leahbriscoe/Documents/MicroBatch/microbatch_vc/data/"
    script_folder = "/Users/leahbriscoe/Documents/MicroBatch/microbatch_vc/RevisionSequence/"

folder = sys.argv[0] # "AGPr_max_k5" #"AGPr_complete_otu" 
trans = sys.argv[1] #"rel"
correction= sys.argv[2] #"rel"
data_dir = main_dir + folder + "/"
d1 = 'FengQ_2015'
d2 = 'ZellerG_2014'


In [13]:
metadata_table = pd.read_csv(data_dir + "metadata.txt", delimiter = "\t",header=0)

if correction == "":
    feature_table = pd.read_csv(data_dir  + "feature_table_" + trans + ".txt" ,delimiter = "\t",header=0)
else:
    feature_table = pd.read_csv(data_dir  +"feature_table_" + trans + "_" + correction + ".txt" ,delimiter = "\t",header=0)

In [17]:
control_metadata1 = metadata_table[(metadata_table['bin_crc_normal'] == 0 ) & (metadata_table['dataset_name'] ==  d1) ]
case_metadata1 = metadata_table[(metadata_table['bin_crc_normal'] == 1) & (metadata_table['dataset_name'] ==  d1)]

control_metadata2 = metadata_table[(metadata_table['bin_crc_normal'] == 0 ) & (metadata_table['dataset_name'] ==  d2) ]
case_metadata2 = metadata_table[(metadata_table['bin_crc_normal'] == 1) & (metadata_table['dataset_name'] ==  d2)]

control_samples_d1 = control_metadata1.index
case_samples_d1 = case_metadata1.index

control_samples_d2 = control_metadata1.index
case_samples_d1 = case_metadata1.index

In [21]:
Counter(metadata_table['dataset_name'])

Counter({'FengQ_2015': 154,
         'HanniganGD_2017': 82,
         'ThomasAM_2018a': 80,
         'ThomasAM_2018b': 60,
         'VogtmannE_2016': 110,
         'YuJ_2015': 128,
         'ZellerG_2014': 199})